# Configuring Antelope Catalog to use local ecoinvent
When you first install the `variability_jlca` repo, it will not come with any Ecoinvent data. Instead, you will have to install your local copy.  That's done using the `EcoinventConfig` class.  This class requires that the ecoinvent data be stored in a directory structure consistent with the way files are distributed by ecoinvent.

In [1]:
import os
from antelope_catalog import LcCatalog

STUDY_PATH = '/data/GitHub/2019/variability_jlca/'

cat = LcCatalog(os.path.join(STUDY_PATH,'catalog'), quell_biogenic_co2=True)

Loading JSON data from /data/GitHub/lca-tools/lcatools/qdb/data/flowable_synlist.json:
Loading JSON data from /data/GitHub/lca-tools/lcatools/qdb/data/quantity_synlist.json:
local.qdb: /data/GitHub/2019/variability_jlca/catalog/reference-quantities.json
Loading JSON data from /data/GitHub/2019/variability_jlca/catalog/reference-quantities.json:
local.qdb: /data/GitHub/lca-tools/lcatools/qdb/data/elcd_reference_quantities.json
25 new quantity entities added (25 total)
6 new flow entities added (6 total)


## Step 1: Install Ecoinvent 3.4

In [2]:
from antelope_catalog.data_sources.ecoinvent import EcoinventConfig

In order to use `EcoinventConfig`, the only required input is the path to the ecoinvent LCI data.  This path should include subfolders named by minor version, e.g. `3.2`, `3.3`, `3.4`.  Those subfolders should include the authentic `.7z` files downloaded from the ecoinvent website, e.g. `ecoinvent 3.4_apos_ecoSpold02.7z`. 

With old versions, the `7z` file could be accessed directly; however, beginning with version 3.4, the Ecoinvent Centre began using an up-to-date `7zip` algorithm to which python's `pylzma` library has not yet caught up, and so these files need to be uncompressed before they can be accessed.  Using the default settings, the uncompressed files will go into a directory with the same name as the `7z` file, e.g. `ecoinvent 3.4_apos_ecospold02`.  The `EcoinventConfig` class will already expect a subfolder called `datasets` to contain the actual XML files.

Ecoinvent's LCIA files should either be placed in their appropriate subdirectories, or in another directory alongside the minor versions whose name is case-insensitively equal to `lcia`.

In [3]:
# this will create the configurator and list the resources it can find
ec = EcoinventConfig('/data/LCI/Ecoinvent')
for x in ec.references:
    print(x)

local.ecoinvent.3.4.apos
local.ecoinvent.3.4.conseq
local.ecoinvent.3.4.cutoff
local.ecoinvent.3.2.apos
local.ecoinvent.3.2.conseq
local.ecoinvent.3.2.cutoff
local.lcia.ecoinvent.3.1
local.ecoinvent.3.3.cutoff


In [4]:
for ref in ec.references:
    if ref.startswith('local.ecoinvent.3.4'):
        for res in ec.make_resources(ref):
            cat.add_resource(res)

After adding the resources, the catalog will list them in its interfaces.  The resources still need to be loaded, indexed, and used to generate (static) background LCI matrices.

In [5]:
cat.show_interfaces()

local.ecoinvent.3.4.apos [basic, inventory]
local.ecoinvent.3.4.conseq [basic, inventory]
local.ecoinvent.3.4.cutoff [basic, inventory]
local.qdb [basic, index, quantity]


In [6]:
# trigger data source index + background creation
cat.query('local.ecoinvent.3.4.cutoff').check_bg()

local.ecoinvent.3.4.cutoff: /data/LCI/Ecoinvent/3.4/ecoinvent 3.4_cutoff_ecoSpold02
Loading /data/LCI/Ecoinvent/3.4/ecoinvent 3.4_cutoff_ecoSpold02
Squashing bad termination in linked reference exchange, 160586f0-0cfa-4f16-8b3a-9e6b9aae4e6e
Flow c5651546-a49e-4c1d-b856-1cf358ab466a Term 66962eb8-0962-43f0-bdbe-634d2ba63cfd
Squashing bad termination in linked reference exchange, a041684a-b77d-448e-b82b-d74610e7ab65
Flow aa447135-9cc5-4f34-9385-ca2a0ae70d3c Term e99cb878-812a-4019-aeac-6e2cf68b8e55
 Loaded 100 processes (t=1.18 s)
 Loaded 200 processes (t=3.24 s)
 Loaded 300 processes (t=4.36 s)
 Loaded 400 processes (t=5.51 s)
 Loaded 500 processes (t=6.27 s)
 Loaded 600 processes (t=7.08 s)
Squashing bad termination in linked reference exchange, 9ba6a249-6003-47b6-92e6-cc6cec625030
Flow c5651546-a49e-4c1d-b856-1cf358ab466a Term abe2477a-5f31-458e-a3aa-92928702c077
 Loaded 700 processes (t=9.36 s)
 Loaded 800 processes (t=11.79 s)
Squashing bad termination in linked reference exchange, 

 Loaded 6000 processes (t=66.81 s)
Squashing bad termination in linked reference exchange, d8449404-cafc-4b42-b159-e2a50b2f7433
Flow aa447135-9cc5-4f34-9385-ca2a0ae70d3c Term aa948b8d-c846-4661-8762-8f0a43f5730d
 Loaded 6100 processes (t=67.76 s)
Squashing bad termination in linked reference exchange, af502a7a-c459-4eaf-b34b-74722dafc781
Flow aa447135-9cc5-4f34-9385-ca2a0ae70d3c Term e99cb878-812a-4019-aeac-6e2cf68b8e55
 Loaded 6200 processes (t=68.48 s)
 Loaded 6300 processes (t=69.64 s)
 Loaded 6400 processes (t=70.56 s)
 Loaded 6500 processes (t=71.14 s)
 Loaded 6600 processes (t=71.73 s)
 Loaded 6700 processes (t=72.35 s)
 Loaded 6800 processes (t=73.11 s)
 Loaded 6900 processes (t=73.81 s)
 Loaded 7000 processes (t=74.36 s)
Squashing bad termination in linked reference exchange, 71663e8e-124a-49ba-9402-60406dc01321
Flow df12f8f5-a3e3-4506-9d86-b33205d21a52 Term 95010049-ec26-4567-8381-86f3b3042802
Squashing bad termination in linked reference exchange, 71663e8e-124a-49ba-9402-6040

True

In [7]:
# repeat for APOS-- slower bc of my sad, memory constrained machine
cat.query('local.ecoinvent.3.4.apos').check_bg()

local.ecoinvent.3.4.apos: /data/LCI/Ecoinvent/3.4/ecoinvent 3.4_apos_ecoSpold02
Loading /data/LCI/Ecoinvent/3.4/ecoinvent 3.4_apos_ecoSpold02
 Loaded 100 processes (t=2.12 s)
 Loaded 200 processes (t=4.35 s)
 Loaded 300 processes (t=5.65 s)
 Loaded 400 processes (t=8.13 s)
 Loaded 500 processes (t=9.23 s)
 Loaded 600 processes (t=11.97 s)
 Loaded 700 processes (t=15.44 s)
 Loaded 800 processes (t=16.82 s)
 Loaded 900 processes (t=18.79 s)
 Loaded 1000 processes (t=21.81 s)
 Loaded 1100 processes (t=22.85 s)
 Loaded 1200 processes (t=24.09 s)
 Loaded 1300 processes (t=26.35 s)
 Loaded 1400 processes (t=28.95 s)
 Loaded 1500 processes (t=32.22 s)
 Loaded 1600 processes (t=35.12 s)
 Loaded 1700 processes (t=36.59 s)
 Loaded 1800 processes (t=38.20 s)
 Loaded 1900 processes (t=39.23 s)
 Loaded 2000 processes (t=40.89 s)
 Loaded 2100 processes (t=44.25 s)
 Loaded 2200 processes (t=46.46 s)
 Loaded 2300 processes (t=47.78 s)
 Loaded 2400 processes (t=49.58 s)
 Loaded 2500 processes (t=51.64 

True

## LCIA
For LCIA, we simply use Ecoinvent's LCIA factors for simplicity.  Assuming these are found by the 

In [8]:
ei_lcia = next(ec.make_resources('local.lcia.ecoinvent.3.1'))
cat.add_resource(ei_lcia)

In [9]:
cat.show_interfaces()

local.ecoinvent.3.4.apos [basic, inventory]
local.ecoinvent.3.4.apos.index.20190204 [background, basic, index]
local.ecoinvent.3.4.conseq [basic, inventory]
local.ecoinvent.3.4.cutoff [basic, inventory]
local.ecoinvent.3.4.cutoff.index.20190204 [background, basic, index]
local.lcia.ecoinvent.3.1 [basic, index, quantity]
local.qdb [basic, index, quantity]


In [10]:
ei_lcia

LcResource(local.lcia.ecoinvent.3.1, dataSource=/data/LCI/Ecoinvent/LCIA/LCIA implementation v3.1 2014_08_13.xlsx:EcoinventLcia, ['quantity', 'index', 'basic'] [50])

The Ecoinvent LCIA resource is considered "static" because it must be loaded all at once (the XLS file cannot be 'partially' opened). So, we load it once and cache the results in the catalog (as a JSON file- no fancy database backends as yet) to cut the load time from ~40 sec to ~2.

In [11]:
cat.create_source_cache(ei_lcia.source, static=True)  # avoid having to load the XLS every time

local.lcia.ecoinvent.3.1: /data/LCI/Ecoinvent/LCIA/LCIA implementation v3.1 2014_08_13.xlsx
Loading /data/LCI/Ecoinvent/LCIA/LCIA implementation v3.1 2014_08_13.xlsx
702 new quantity entities added (702 total)
3255 new flow entities added (3255 total)
local.lcia.ecoinvent.3.1: /data/GitHub/2019/variability_jlca/catalog/cache/cfa3982d248fbdf0224e3ebad316063c182bbbbd.json.gz
Created archive of EcoinventLcia with 3957 entities at /data/LCI/Ecoinvent/LCIA/LCIA implementation v3.1 2014_08_13.xlsx containing:
0 new quantity entities added (702 total)
0 new flow entities added (3255 total)
